Install the Unsloth Python package.

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Load the model and tokenizer using the FastLanguageModel function.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Yellow-AI-NLP/komodo-7b-base",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "hf_**********", # Get a token at https://huggingface.co/settings/tokens
)

We add the LoRA (Low-Rank Adapter) to the model using the linear modules from the base model.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Creating a prompt style to help achieve the desired results. A prompt consists of instructions, input, and output.

In [ ]:
prompt_style = """Di bawah ini terdapat sebuah instruksi yang menggambarkan sebuah tugas, disertai dengan sebuah masukan yang memberikan konteks lebih lanjut. Tulislah respons yang sesuai untuk menyelesaikan permintaan tersebut.

### Instruksi:
Anda adalah seorang asisten virtual Indonesia. Tugas Anda adalah memberikan penjelasan yang jelas dan ringkas berdasarkan pengetahuan anda. Pastikan respons Anda akurat, objektif, mudah dipahami, dan menggunakan bahasa Indonesia.

### Masukan:
{}

### Respons:
{}"""

In [ ]:
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output)
        texts.append(text)
    return { "text" : texts, }

Load the dataset and apply the predefined prompt format to it.








In [ ]:
from datasets import load_dataset

# dataset = load_dataset("csv", data_files="dataset-finetuning/merged_dataset.csv", split="train[0:50000]") # If your data from csv files

dataset = load_dataset("Ichsan2895/alpaca-gpt4-indonesian", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][0]

Set up the trainer with the model, tokenizer, max sequence length, and training arguments.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Train the model

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

The output of the training is a LoRA adapter that can be saved locally.

In [ ]:
# Local Saving
model.save_pretrained("/unsloth-komodo-7B-v2")
tokenizer.save_pretrained("/unsloth-komodo-7B-v2")

You can also merge the adapter with the base model and save it locally.

In [2]:
model.save_pretrained_merged("/unsloth-komodo-7B-v2", tokenizer, save_method = "merged_16bit",)

You can also push the adapter model or the merged model to Hugging Face.

In [ ]:
model.push_to_hub_merged("digo-prayudha/unsloth-komodo-7B-v2", tokenizer, save_method = "merged_16bit", token = "hf_*******") # Merged Model
model.push_to_hub_merged("digo-prayudha/unsloth-komodo-7B-v2", tokenizer, save_method = "lora", token = "hf_*******") # Adapter Model

You can also save it in the GGUF format.

In [ ]:
model.push_to_hub_gguf(
    "unsloth-llama-3.2-1b-gguf", # Change hf to your username!
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
    token = "hf_*********",
)